In [18]:
# load basics library 
import pandas as pd, numpy as np

# import pandasql library to work with dataframe with SQL 
from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

In [35]:
# example df
# random integer >= 1 and <= 20, with 5 counts 

#name=['']
df = pd.DataFrame({'name':['jim','sam','amy','liz','jane',
                          'jim','sam','amy','liz','jane'],
                   'point':np.random.randint(1,20,10),
                   'cash':np.random.randint(1,20,10),
                   'pay':np.random.randint(1,20,10)} )
df= df[['name','cash','pay','point']]

In [95]:
df.head(3)

,name,cash,pay,point
0,jim,6,4,14
1,sam,17,18,15
2,amy,4,10,5


### 1)  product all value on every category  


In [94]:
# http://www.itpub.net/thread-1577037-1-1.html

# METHOD 1 : via SQL 
SQL2="""

SELECT name,
       exp(s_log_cash) AS sum_cash
FROM
  ( SELECT name,
           sum(log(cash)) AS s_log_cash
   FROM df
   GROUP BY 1) sub

"""

In [90]:
# METHOD 2 : via pandas 

def get_log(x):
    return np.log(x)

def get_exp(x):
    return np.exp(x)


# group over index 
df.set_index('name').apply(get_log).groupby(level=0).sum().apply(get_exp)

,cash,pay,point
name,,,
amy,32.0,30.0,20.0
jane,26.0,18.0,105.0
jim,36.0,52.0,182.0
liz,81.0,238.0,64.0
sam,204.0,180.0,120.0


In [99]:
df.set_index('name').apply(lambda x : np.log(x))\
                    .groupby(level=0).sum()\
                    .apply(lambda x : np.exp(x))

,cash,pay,point
name,,,
amy,32.0,30.0,20.0
jane,26.0,18.0,105.0
jim,36.0,52.0,182.0
liz,81.0,238.0,64.0
sam,204.0,180.0,120.0
